# Calculation of turbulence boundary conditions

Using the $k-\epsilon$ model one need to specify the inlet value for such quantites. Given the $Re$
$$\begin{align*}
k = \frac{1}{2}{\bf U}^{\prime} \cdot {\bf U}^{\prime}
\\
\epsilon = C_{\mu}^{0.75} \frac{k^{3/2}}{l_t}
\end{align*}$$
and
$$\begin{equation*}
\nu_t = C_{\mu} \frac{k^2}{\epsilon}
\end{equation*}$$

$\nu_t$ can be calcaulated from  the turbulence intensity and length scale
$$\nu_t = \sqrt{\frac{3}{2}} \left( uIl_t\right )$$

Another importamt quantity to set the time step $\Delta t$, that can be computed based on Courant Number $Co$

$$\Delta t = Co \frac{\Delta x}{u}$$

Another parameter is Acoustic Co number defined as

$$Co_{acoustic} = \frac{\Delta t}{\Delta x \sqrt{\psi}}$$

On the same lines $\omega$ is give as
$$\omega = \frac{C_{\mu}\sqrt{k}}{l_t}$$


$$\omega_{wall} = 10\frac{6 \nu}{\beta_1 (\Delta d_1)^2}$$

where $\beta_1 = 0.075$, where $\Delta d_1$ is the distance to the next point away from the wall.


While $I$  is the initial turbulence intensity [%] can be computed as
$$I=0.16Re^{-{\frac {1}{8}}}$$

In compressible and heat transfer problems, one also need to specify the turbulent Prandl number $Pr_t$. Generally it can be assumed 0.7 to 0.85. A better relations for liquids is given as
$$Pr_t = 6.374Re^{-0.238} Pr^{-0.161}$$
In case of compressibleInterFoam solver, we need to specify alphat $\alpha_t$, which is given by
$$\alpha_t = \frac{\nu_t}{Pr_t}$$

# Calculating first grid point based on yPlus

The yPlus relation is given by
$$y^+ = \Delta s \frac{u^* \mu}{\rho} $$
where $\Delta s$ is the distance to first grid point. $u^*$ is the friction velocity explained below. Given a $y^+$ we can estimate the $\Delta s$ as
$$\Delta s = \frac{y^+ \mu}{u^* \rho}$$

## yPlus from WolfDynamics
Compute skin friction coefficeint $C_f$

For plates we can write
$$C_f = 0.058 \times Re^{-0.2}$$
Then calculate the wall shear stress $\tau_w$
$$\tau_w = \frac{1}{2} \times C_f \times \rho \times U^2_{\infty} $$
The calculate the friction velocity $U_\tau$
$$U_{\tau} = \sqrt{\frac{\tau_w}{\rho}}$$
Finally get the distance $y$ to have a specified $y^+$
$$y = \frac{\mu \times y^+ }{\rho \times U_{\tau}}$$

# Using the applyBoundaryLayer pre-processing to apply 1/7 power law profile

For such a case, one need to specify -ybl value, which is the with of viscous sub layer ($\delta_{vsc}$), which can be computed using the friction velocity $u^*$, which in turn can be computed using the $\tau_w$. $\tau_w$ can be estimated using an approximation of friction factor $f$. Several relations for pipe flow friction factor are avaialable based on $Re$. Forexample,
$$ f = 0.3164/Re^{0.25} \ for \ Re<10^5$$
and based on the wall roughness $\epsilon_{wall}$, a rough estimate within $\pm2\%$ is
$$ \frac{1}{\sqrt{f}} =-1.8 \log_{10} \left[ 6.9/Re + \left(\frac{\epsilon_{wall}/D}{3.7}\right)^{1.11} \right]$$

The wall shear stress can be computed as
$$\tau_w = \frac{1}{4}f \frac{\rho V^2}{2} = C_f \frac{\rho V^2}{2}$$
The $C_f$ is the average skin friction coefficient and some expressions are reported at 
https://www.cfd-online.com/Wiki/Skin_friction_coefficient

The friction velocity is then computed as
$$ u^* = \left( \frac{\tau_w}{\rho} \right)^{1/2} $$

Finally, the viscous sub layer thickness computed as
$$ \delta_{vsl} = 5 \frac{\nu}{u^*}$$

The Taylor based length scale $\lambda$ can be computed as
$$\lambda_{T} = D_h \sqrt{10} Re^{-1/2}$$

The viscous length scale for pipe can be computed also as [1] (to be checked from a another refererence)
$$l_v = 5.0 Re^{-7/8}$$
and a Kolmogorov critical radius $l_{cr}$ as
$$l_{cr} = \left(\frac{\sigma^3}{\rho^3 \epsilon^2}\right)^{1/5}$$
[1]  @techreport{bravo2015high,
  title={High fidelity simulation of atomization in diesel engine sprays},
  author={Bravo, L and Ivey, CB and Kim, D and Bose, ST},
  year={2015},
  institution={ARMY RESEARCH LAB ABERDEEN PROVING GROUND MD VEHICLE TECHNOLOGY DIRECTORATE}
}

## Get the density and other transport properties

In [1]:
import numpy as np
from thermo.chemical import Chemical
from CoolProp.CoolProp import PropsSI

## Input data
H = 4e-3 #
Din = 1e-3
A = Din*H
P = 2*(Din+H)
Dh = 4*A/P
print("Dh Calculated", Dh , "[m]")
# Dh manual entry
Dh = H
print("Dh Manual", Dh , "[m]")

## Mass flow rate
mf = 2.4/60
print("mf", mf, "[kg/s]")

## Get the fluid properties
fluidName = "Water"
T_fluid = 293  # temperature
P_fluid = 1.01325e5  # pressure

fluid = Chemical(fluidName)
fluid.calculate(T=T_fluid, P=P_fluid)
print("fluid:", fluidName)
rho = fluid.rho
print("rho from Thermo package", rho , "[kg/m^3]")
# get density from CoolProp
rho = PropsSI('D','T',T_fluid,'P',P_fluid,fluidName)
print("rho", rho , "[kg/m^3]")
mu = PropsSI('V','T',T_fluid,'P',P_fluid,fluidName)
print("mu", "{:.3e}".format(mu), "[Pa-s] or [Kg/m/s]")
nu = mu/rho
print("nu", "{:.3e}".format(nu), "[m^2/s]")
Pr = PropsSI('Prandtl','T',T_fluid,'P',P_fluid,fluidName)
print("Pr (Cp mu/K)", "{:.3e}".format(Pr))


#Saturation properties
#Saturation temperature of fluid at P_fluid pressure
satT = PropsSI('T','P',P_fluid,'Q',0,fluidName)
print("satT at P[Pa] =",P_fluid, "is" , "{:.3f}".format(satT), "[K]")

#Saturation pressure of fluid at T_fluid temperature
satP = PropsSI('P','T',T_fluid,'Q',0,fluidName)
print("satP at T[K] =",T_fluid, "is" ,"{:.3f}".format(satP), "[Pa]")

#Saturated vapor density of fluid at T_fluid
satRhoVap = PropsSI('D','T',T_fluid,'Q',1,fluidName)
print("satRhoVap", "{:.3f}".format(satRhoVap), "[kg/m3]")

#Sat vapor viscosity at given temperature
muVap = PropsSI('V','T',T_fluid,'Q',1,fluidName)
print("muVap", "{:.3e}".format(muVap), "[Pa-s] or [Kg/m/s]")
nuVap = muVap/satRhoVap
print("nuVap", "{:.3e}".format(nuVap), "[m^2/s]")

Ux = mf/(A*rho) # also need the actual reduced area
print("Ux", Ux, "[m/s]")
Ux = 16
print("Ux Manual Input", Ux, "[m/s]")
Re = rho*Ux*Dh/mu
print("Re (based on Dh) =", Re)

Re = rho*Ux*Din/mu
print("Re (based on Width) =", Re)

#rho = 1000
#mu = 1e-5
Re = rho*Ux*H/mu
print("Re (based on Height) =", Re)

Dh = H
# Taylor based length scale
lambda_T = Dh*(10**0.5)*Re**(-0.5)
print("lambda_T", lambda_T,  "[m]")

Dh Calculated 0.0015999999999999999 [m]
Dh Manual 0.004 [m]
mf 0.04 [kg/s]
fluid: Water
rho from Thermo package 998.2386619948825 [kg/m^3]
rho 998.2379970668911 [kg/m^3]
mu 1.005e-03 [Pa-s] or [Kg/m/s]
nu 1.007e-06 [m^2/s]
Pr (Cp mu/K) 7.037e+00
satT at P[Pa] = 101325.0 is 373.124 [K]
satP at T[K] = 293 is 2317.670 [Pa]
satRhoVap 0.017 [kg/m3]
muVap 9.539e-06 [Pa-s] or [Kg/m/s]
nuVap 5.558e-04 [m^2/s]
Ux 10.017651130675112 [m/s]
Ux Manual Input 16 [m/s]
Re (based on Dh) = 63551.22093297681
Re (based on Width) = 15887.805233244202
Re (based on Height) = 63551.22093297681
lambda_T 5.017623169187479e-05 [m]


In [2]:
## print Re
print("Re",Re)

## Friction Factor
f = 0.3164/Re**0.25
print("f =", f)
# Simplified Cf for plate
Cf = 0.058/Re**0.2
print("Cf =", Cf)

tau_w = 0.25*f*0.5*rho*Ux**2
print("tau_w", tau_w)
tau_w_Cf = 0.5*Cf*rho*Ux**2
print("tau_w_Cf", tau_w_Cf)

uTau = (tau_w_Cf/rho)**0.5
print("uTau", uTau)
uStar = pow(tau_w/rho,0.5)
uStar = (tau_w/rho)**0.5
print("uStar", uStar)
delta_vsl = 5*nu/uStar
delta_vsl_Cf = 5*nu/uTau
print("viscous sub layer thickness =", delta_vsl)
print("viscous sub layer thickness Cf =", delta_vsl_Cf)
lv = 5.0*pow(Re,(-7.0/8.0))
print("lv", lv)

## Cell point bsaed on yPlus
yPlus = 1.0
firstCellHeight = yPlus*mu/(uStar*rho)
firstCellHeight_Cf = yPlus*mu/(uTau*rho)
print('yPlus = ', yPlus, "requires first cell height [m]", "{:.3e}".format(firstCellHeight))
print('yPlus = ', yPlus, "requires first cell height [m] with Cf ", "{:.3e}".format(firstCellHeight_Cf))

Re 63551.22093297681
f = 0.019927623081078805
Cf = 0.006350431179146555
tau_w 636.5603376243218
tau_w_Cf 811.4229377001463
uTau 0.901584821816982
uStar 0.7985511496419762
viscous sub layer thickness = 6.30555532460374e-06
viscous sub layer thickness Cf = 5.584952554376019e-06
lv 0.00031349931274718385
yPlus =  1.0 requires first cell height [m] 1.261e-06
yPlus =  1.0 requires first cell height [m] with Cf  1.117e-06


In [3]:
lt_scale = 5 # turbulent lenght scale
print("Din", Din)
lt = Din*lt_scale/100
print("mixingLength", lt)
It = 5 # turbulence intensity
UxPrime = Ux*It/100
print("UxPrime", UxPrime)
k = 0.5*(UxPrime**2)
print("k", k,  "[m^2/s^2]")
epsilon = 0.09**0.75*k**1.5/lt
print("epsilon", '{0:e}'.format(epsilon), "[m^2/s^3]")
nut = 0.09*k**2/epsilon
print("nut", nut,  "[m^2/s]")
omega = 0.09**(-0.25)*k**0.5/lt
print("omega", '{0:e}'.format(omega), "[1/s]")
deltaD1 = 89.4/20 #distanToWall
deltaD1 = 3.5e-6
omega_wall = 6*nu/(0.075*deltaD1**2)
print("omega_wall", '{0:e}'.format(omega_wall))



Co = 0.75
nyCells = 10
DeltaX = Din/10
DeltaX = 5e-6
DeltaT = Co*DeltaX/Ux
print("DeltaT", DeltaT)
psi_l = 5e-7
psi_v = 2.5e-6
CoAc = 12
DeltaT_CoAc = DeltaX*(psi_l**0.5)
print("DeltaT_CoAc", DeltaT_CoAc)

# Prandtl number for fluid $$Pr_t = 6.374Re^{-0.238} Pr^{-0.161}$$
Pr_l =  13
Pr_l = Pr # from coolprop
print("Prandtl Number ", Pr)
Prt_l = 6.374*pow(Re,-0.238)*pow(Pr_l,-0.161)
print("Turbulent Prandtl Number Prt ", Prt_l)
mut = nut*rho
print("mut = ", mut, "[Pa-s] or [Kg/m/s]")
alphat = mut/Prt_l
print("alphat (mut/Prt), Prt =",Prt_l, "gives", '{0:e}'.format(alphat), "[Pa-s] or [Kg/m/s]")
alphat = mut/0.7
print("alphat based on Prt=0.7 is", '{0:e}'.format(alphat), "[Pa-s] or [Kg/m/s]")

Din 0.001
mixingLength 5e-05
UxPrime 0.8
k 0.32000000000000006 [m^2/s^2]
epsilon 5.948902e+02 [m^2/s^3]
nut 1.549193338482967e-05 [m^2/s]
omega 2.065591e+04 [1/s]
omega_wall 6.576729e+06
DeltaT 2.3437500000000003e-07
DeltaT_CoAc 3.535533905932738e-09
Prandtl Number  7.036891065061594
Turbulent Prandtl Number Prt  0.3348422549635111
mut =  0.015464636552766073 [Pa-s] or [Kg/m/s]
alphat (mut/Prt), Prt = 0.3348422549635111 gives 4.618484e-02 [Pa-s] or [Kg/m/s]
alphat based on Prt=0.7 is 2.209234e-02 [Pa-s] or [Kg/m/s]


# Setting the expansion Ratio of cells in blockMesh

$R$ is the final to first cell size ratio. 
$r$ is the first to second expansion ratio.
$n$ is the number of cells
$\delta x_s$ is the smallest cell size
$l$ is the total length. Then
$$r = R^{\frac{1}{n-1}}$$
$$\delta x_s = l \frac{r-1}{\alpha r - 1}$$
and
$$ \alpha = R \ for \ R>1$$

\begin{equation*}
	\alpha = 1-r^{-n} + r^{-1} \ for \ R<1
\end{equation*}

$$ r = \frac{l - \delta x_s}{l - \delta x_s R}$$
$$n = \frac{\ln R}{\ln r}+1$$

Setting the double expansion ration given as

(0.05 0.11 10)    // 20% x-dir, 30% cells, expansion = 4 

(0.9 0.78 1)    // 60% x-dir, 40% cells, expansion = 1 

(0.05 0.11 0.1) // 20% x-dir, 30% cells, expansion = 0.25 (1/4) 